# Image generation tutorial

* Create package objects required for image generation
* Generate noisy sample images

In [ ]:
import state_reconstruction as srec

import numpy as np
import os
import PIL

from libics.env import DIR_DESKTOP
from libics.core import io
from libics.tools import plot
from libics.tools.trafo.linear import AffineTrafo2d

## Package configuration

**The `state_reconstruction` package configuration**

* can be read as follows
* and can be altered by overwriting the appropriate parameters in the configuration file located at `~/.libics/state_estimation/config.json`, where `~` indicates the user folder (e.g. `C:/Users/<my_user_name>`

In [ ]:
srec.get_config()

**Files generated using this tutorial**

* are saved to the desktop with the following file name
* and may be used by other tutorials

In [ ]:
DEMO_FILENAME = "srec_demo"

## Transformation between image and sites

**Generate 2D affine transformation**

* The coordinate systems of the image pixels and the lattice sites are related by a two-dimensional affine transformation
* A systematic way to parameterizing the transformation consists of specifying magnification, rotation and offset of the coordinate axes
* The offset can be conveniently specifying by defining a point in each coordinate system, that should be mapped onto each other

In [ ]:
magnification = 4.25
angle = 45
site_ref = srec.get_config("trafo_gen.phase_ref_site")
image_ref = srec.get_config("trafo_gen.phase_ref_image")
print("site_ref:", site_ref)
print("image_ref:", image_ref)

trafo_site_to_image = AffineTrafo2d()
# Set site unit vectors within image coordinate system
trafo_site_to_image.set_origin_axes(
    magnification=[magnification, magnification],
    angle=np.deg2rad([angle, angle])
)
trafo_site_to_image.set_offset_by_point_pair(
    site_ref, image_ref
)
trafo_site_to_image

In [ ]:
# Example for affine transformation of an image
_site_shape = np.array(site_ref) * 2
_image_shape = np.array(image_ref) * 2

_site_ar = np.arange(np.prod(_site_shape)).reshape(_site_shape)
_image_ar = trafo_site_to_image.cv_to_target(
    _site_ar, _image_shape
)

fig, axs = plot.subplots(figsize=(8, 3), ncols=2)
plt_params = dict(
    cmap="viridis", colorbar=True, vmin=0, vmax=np.max(_site_ar)
)
plot.pcolorim(_site_ar, ax=axs[0], title="Site coordinate system", **plt_params)
plot.pcolorim(_image_ar, ax=axs[1], title="Image coordinate system", **plt_params)
plot.style_figure(tight_layout=True)

In [ ]:
# Save trafo object to json file
io.save(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_trafo.json"),
    trafo_site_to_image
)

In [ ]:
# For future use, the trafo object can be directly loaded from file
trafo_site_to_image = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_trafo.json")
)
trafo_site_to_image

## Point spread functions

**Generate PSF**

* We first create PSFs as arrays (with shape `psf_size`)
* These may have higher resolution than the images (with supersampling factor `psf_supersample`)
* There are built-in functions for PSFs with Gaussian or Airy form

In [ ]:
wavelength = 780e-9
lattice_spacing = 532e-9
numerical_aperture = 0.68
psf_size = (21, 21)
psf_supersample = 5

_trafo_magnification = trafo_site_to_image.get_origin_axes()[0].mean()
px_size = lattice_spacing / _trafo_magnification
gaussian_width = srec.get_psf_gaussian_width(
    wavelength, px_size=px_size,
    numerical_aperture=numerical_aperture
)
gaussian_psf = srec.get_psf_gaussian(
    wx=psf_supersample*gaussian_width,
    size=psf_supersample*np.array(psf_size)
)

airy_width = srec.get_psf_airy_width(
    wavelength, px_size=px_size,
    numerical_aperture=numerical_aperture
)
airy_psf = srec.get_psf_airy(
    wx=psf_supersample*airy_width,
    size=psf_supersample*np.array(psf_size)
)

fig, axs = plot.subplots(figsize=(8, 3), ncols=2)
plt_params = dict(
    cmap="viridis", colorbar=True, vmin=0
)
plot.pcolorim(gaussian_psf, ax=axs[0], title="Gaussian PSF", **plt_params)
plot.pcolorim(airy_psf, ax=axs[1], title="Airy PSF", **plt_params)
plot.style_figure(tight_layout=True)

**Create `state_reconstruction` PSF object**

In [ ]:
ipsf_gen = srec.IntegratedPsfGenerator(
    psf=airy_psf, psf_supersample=psf_supersample
)
ipsf_gen

In [ ]:
# Save the PSF object to a json file
ipsf_gen.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_psf.json"))

In [ ]:
# For future use, the PSF object can be directly loaded from file
ipsf_gen = srec.IntegratedPsfGenerator.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_psf.json")
)
ipsf_gen

## Images

* Create `state_reconstruction` image generator object

**Generate dense noisy images**

In [ ]:
image_size = (512, 512)      # Shape of images
sites_size = (170, 170)      # Shape of reconstructed sites

img_gen = srec.ImageGenerator(
    image_size=image_size,
    sites_size=sites_size,
    trafo_site_to_image=trafo_site_to_image,
    psf=ipsf_gen.psf,
    psf_supersample=ipsf_gen.psf_supersample
)
img_gen

In [ ]:
lattice_phase = (0.0, 0.0)  # 2D lattice phases in units of lattice spacing
rng_seed = 0

_res = img_gen.generate_image(
    seed=rng_seed,
    sites_phase=lattice_phase,
    phase_ref_image=srec.get_config("trafo_gen.phase_ref_image"),
    phase_ref_site=srec.get_config("trafo_gen.phase_ref_site"),
    ret_vals=["occ_2d_sites"]
)
demo_occs = _res["occ_2d_sites"]
demo_image = _res["image_sample"]
# Convert to positive integer values
demo_image += 100
demo_image[demo_image < 0] = 0
demo_image.data = np.array(np.round(demo_image.data), dtype="uint16")

plot.pcolorim(demo_image, cmap="hot", colorbar=True, vmin=100)

In [ ]:
# Save demo image to a png file
pil_image = np.array(demo_image, dtype="uint16").T
pil_image = PIL.Image.fromarray(pil_image)
pil_image.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-dense.png"))
# Save occupation to a json file
io.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_occ-dense.json"), demo_occs)

In [ ]:
# Loaded the image from file
demo_image = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-dense.png")
)
plot.pcolorim(demo_image, cmap="hot", colorbar=True, vmin=100)

**Generate sparse noisy images**

In [ ]:
image_size = (512, 512)      # Shape of images
sites_size = (170, 170)      # Shape of reconstructed sites

img_gen = srec.ImageGenerator(
    image_size=image_size,
    sites_size=sites_size,
    atoms_size=(75, 75),
    atoms_filling=0.015,
    trafo_site_to_image=trafo_site_to_image,
    psf=ipsf_gen.psf,
    psf_supersample=ipsf_gen.psf_supersample
)
img_gen

In [ ]:
lattice_phase = (0.0, 0.0)  # 2D lattice phases in units of lattice spacing
rng_seed = 0

_res = img_gen.generate_image(
    seed=rng_seed,
    sites_phase=lattice_phase,
    phase_ref_image=srec.get_config("trafo_gen.phase_ref_image"),
    phase_ref_site=srec.get_config("trafo_gen.phase_ref_site"),
    ret_vals=["occ_2d_sites"]
)
demo_occs = _res["occ_2d_sites"]
demo_image = _res["image_sample"]
# Convert to positive integer values
demo_image += 100
demo_image[demo_image < 0] = 0
demo_image.data = np.array(np.round(demo_image.data), dtype="uint16")

plot.pcolorim(demo_image, cmap="hot", colorbar=True, vmin=100)

In [ ]:
# Save demo image to a png file
pil_image = np.flip(np.array(demo_image, dtype="uint16").T, axis=0)
pil_image = PIL.Image.fromarray(pil_image)
pil_image.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-sparse.png"))
# Save occupation to a json file
io.save(os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_occ-sparse.json"), demo_occs)

In [ ]:
# Loaded the image from file
demo_image = io.load(
    os.path.join(DIR_DESKTOP, DEMO_FILENAME + "_image-sparse.png")
)
plot.pcolorim(demo_image, cmap="hot", colorbar=True, vmin=100)